In [105]:
import datetime
import ast
import json

import numpy as np
import pandas as pd
import string
import re

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestRegressor

#### Read Dataset

In [106]:
df = pd.read_csv('cars.csv')
pd.reset_option('display.max_rows')
df.head()

,link,listing_title,listing_mileage,primary_price,deal_gauge,exterior_color,interior_color,drivetrain,mpg,fuel_type,transmission,engine,vin,stock_number,vehicle_history,seller_name,price_history
0,https://www.cars.com/vehicledetail/5ee3ae04-69...,2018 Hyundai Santa Fe Sport 2.4L,"79,497 mi.","$17,899",Good Deal $2 under,Nightfall Blue,Beige,Front-wheel Drive,21-27,Gasoline,6-Speed Automatic,2.4L I4 16V GDI DOHC,5XYZT3LB3JG542718,13669,"{'Accidents or damage': 'None reported', '1-ow...",ETHIO Motors,"[('9/21/21', 'Listed', '$20,299'), ('2/04/22',..."
1,https://www.cars.com/vehicledetail/c99e5035-76...,2017 RAM 1500 Express,"80,326 mi.","$27,900",This is a good deal.,Silver,Black,Four-wheel Drive,16-23,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,3C6RR7KT1HG533082,KBB2955,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('3/15/22', 'Listed', '$30,985'), ('6/01/22',..."
2,https://www.cars.com/vehicledetail/4e79fa3a-ba...,2019 Nissan Kicks SR,"92,959 mi.","$18,950",This is a fair deal. Why?,Monarch Orange / Super Black,Charcoal,Front-wheel Drive,31-36,Gasoline,Automatic CVT,1.6L I4 16V MPFI DOHC,3N1CP5CU6KL510673,KBB3225,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('4/16/22', 'Listed', '$19,980'), ('6/01/22',..."
3,https://www.cars.com/vehicledetail/86a0d096-32...,2019 RAM 1500 Laramie,"59,176 mi.","$39,950",Great Deal $968 under,Gray,Black,Four-wheel Drive,19-24,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,1C6SRFDT5KN869809,KBB3246,{'Accidents or damage': 'At least 1 accident o...,Custom Cars West,"[('6/01/22', 'Listed', '$40,523'), ('7/14/22',..."
4,https://www.cars.com/vehicledetail/36efbef5-f2...,2015 Jeep Wrangler Unlimited Rubicon,"135,385 mi.","$26,950",This is a good deal.,Blue,Black,Four-wheel Drive,16-21,Gasoline,-,3.6L V6 24V MPFI DOHC,1C4BJWFG2FL530006,KBB3145,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('1/21/22', 'Listed', '$29,950'), ('3/22/22',..."


## Split vehicle_history to 4 parts 
#### Need to adjust this, to more properly deal with dictionaries

In [107]:
# original code, can remove when confirm that it was properly dealt with

#df[['accident','one_owner','personal_use','open_recall']] = df.vehicle_history.str.split(',',3, expand=True)
#df.head()

def extract_dict(s):
    try:
        # Convert the string to a dictionary using literal_eval
        return ast.literal_eval(s)
    except (SyntaxError, ValueError):
        # Handle cases where literal_eval fails, e.g., malformed strings
        return {}

# First convert dtype to str from object
df['vehicle_history'] = df['vehicle_history'].astype(str)
# Apply the extract_dict fcn
df['vehicle_history'] = df['vehicle_history'].apply(extract_dict)
# normalize and join
norm = pd.json_normalize(df['vehicle_history'])
df = pd.concat([df, norm], axis= 1)
#pd.set_option('display.max_rows', None)
#pd.reset_option('display.max_rows')
#df
df.head()

,link,listing_title,listing_mileage,primary_price,deal_gauge,exterior_color,interior_color,drivetrain,mpg,fuel_type,...,vin,stock_number,vehicle_history,seller_name,price_history,Accidents or damage,1-owner vehicle,Personal use only,Open recall,Clean title
0,https://www.cars.com/vehicledetail/5ee3ae04-69...,2018 Hyundai Santa Fe Sport 2.4L,"79,497 mi.","$17,899",Good Deal $2 under,Nightfall Blue,Beige,Front-wheel Drive,21-27,Gasoline,...,5XYZT3LB3JG542718,13669,"{'Accidents or damage': 'None reported', '1-ow...",ETHIO Motors,"[('9/21/21', 'Listed', '$20,299'), ('2/04/22',...",None reported,Yes,No,At least 1 open recall reported,NaN
1,https://www.cars.com/vehicledetail/c99e5035-76...,2017 RAM 1500 Express,"80,326 mi.","$27,900",This is a good deal.,Silver,Black,Four-wheel Drive,16-23,Gasoline,...,3C6RR7KT1HG533082,KBB2955,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('3/15/22', 'Listed', '$30,985'), ('6/01/22',...",None reported,No,Yes,NaN,NaN
2,https://www.cars.com/vehicledetail/4e79fa3a-ba...,2019 Nissan Kicks SR,"92,959 mi.","$18,950",This is a fair deal. Why?,Monarch Orange / Super Black,Charcoal,Front-wheel Drive,31-36,Gasoline,...,3N1CP5CU6KL510673,KBB3225,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('4/16/22', 'Listed', '$19,980'), ('6/01/22',...",None reported,No,Yes,NaN,NaN
3,https://www.cars.com/vehicledetail/86a0d096-32...,2019 RAM 1500 Laramie,"59,176 mi.","$39,950",Great Deal $968 under,Gray,Black,Four-wheel Drive,19-24,Gasoline,...,1C6SRFDT5KN869809,KBB3246,{'Accidents or damage': 'At least 1 accident o...,Custom Cars West,"[('6/01/22', 'Listed', '$40,523'), ('7/14/22',...",At least 1 accident or damage reported,No,No,At least 1 open recall reported,NaN
4,https://www.cars.com/vehicledetail/36efbef5-f2...,2015 Jeep Wrangler Unlimited Rubicon,"135,385 mi.","$26,950",This is a good deal.,Blue,Black,Four-wheel Drive,16-21,Gasoline,...,1C4BJWFG2FL530006,KBB3145,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('1/21/22', 'Listed', '$29,950'), ('3/22/22',...",None reported,No,Yes,At least 1 open recall reported,NaN


In [108]:
df[['year','make','model']] = df.listing_title.str.split(' ',2, expand=True)
df.head()

,link,listing_title,listing_mileage,primary_price,deal_gauge,exterior_color,interior_color,drivetrain,mpg,fuel_type,...,seller_name,price_history,Accidents or damage,1-owner vehicle,Personal use only,Open recall,Clean title,year,make,model
0,https://www.cars.com/vehicledetail/5ee3ae04-69...,2018 Hyundai Santa Fe Sport 2.4L,"79,497 mi.","$17,899",Good Deal $2 under,Nightfall Blue,Beige,Front-wheel Drive,21-27,Gasoline,...,ETHIO Motors,"[('9/21/21', 'Listed', '$20,299'), ('2/04/22',...",None reported,Yes,No,At least 1 open recall reported,NaN,2018,Hyundai,Santa Fe Sport 2.4L
1,https://www.cars.com/vehicledetail/c99e5035-76...,2017 RAM 1500 Express,"80,326 mi.","$27,900",This is a good deal.,Silver,Black,Four-wheel Drive,16-23,Gasoline,...,Custom Cars West,"[('3/15/22', 'Listed', '$30,985'), ('6/01/22',...",None reported,No,Yes,NaN,NaN,2017,RAM,1500 Express
2,https://www.cars.com/vehicledetail/4e79fa3a-ba...,2019 Nissan Kicks SR,"92,959 mi.","$18,950",This is a fair deal. Why?,Monarch Orange / Super Black,Charcoal,Front-wheel Drive,31-36,Gasoline,...,Custom Cars West,"[('4/16/22', 'Listed', '$19,980'), ('6/01/22',...",None reported,No,Yes,NaN,NaN,2019,Nissan,Kicks SR
3,https://www.cars.com/vehicledetail/86a0d096-32...,2019 RAM 1500 Laramie,"59,176 mi.","$39,950",Great Deal $968 under,Gray,Black,Four-wheel Drive,19-24,Gasoline,...,Custom Cars West,"[('6/01/22', 'Listed', '$40,523'), ('7/14/22',...",At least 1 accident or damage reported,No,No,At least 1 open recall reported,NaN,2019,RAM,1500 Laramie
4,https://www.cars.com/vehicledetail/36efbef5-f2...,2015 Jeep Wrangler Unlimited Rubicon,"135,385 mi.","$26,950",This is a good deal.,Blue,Black,Four-wheel Drive,16-21,Gasoline,...,Custom Cars West,"[('1/21/22', 'Listed', '$29,950'), ('3/22/22',...",None reported,No,Yes,At least 1 open recall reported,NaN,2015,Jeep,Wrangler Unlimited Rubicon


In [109]:
#df = df[['year', 'make', 'model', 'listing_mileage', 'primary_price', 'deal_gauge', 'exterior_color', 'interior_color', 'drivetrain', 'mpg', 'fuel_type', 'transmission', 'engine', 'vin', 'stock_number', 'accident', 'one_owner', 'personal_use', 'open_recall', 'seller_name', 'price_history', 'link']]

## Split price_history to listed_date, price_change, price_change_percentage

In [110]:
# Convert string representation of lists to actual lists
df['price_history'] = df['price_history'].apply(ast.literal_eval)

# Extract the oldest date, total price change, and price change percentage
df['listing_date'] = df['price_history'].apply(lambda x: min([pd.to_datetime(item[0], format='%m/%d/%y')for item in x], default=None))

# Calculate total price change and price change percentage
def calculate_changes(prices):
    if not prices:
        return None, None
    prices = [(pd.to_datetime(date, format='%m/%d/%y'), status, int(price.replace('$', '').replace(',', ''))) for date, status, price in prices]
    total_price_change = prices[-1][2] - prices[0][2]
    price_change_percentage = (total_price_change / prices[0][2]) * 100
    return total_price_change, price_change_percentage

df['total_price_change'], df['price_change_percentage'] = zip(*df['price_history'].apply(calculate_changes))
#pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')
#pd.set_option('display.max_rows', None)
#pd.reset_option('display.max_rows')
#df
df.head()

,link,listing_title,listing_mileage,primary_price,deal_gauge,exterior_color,interior_color,drivetrain,mpg,fuel_type,...,1-owner vehicle,Personal use only,Open recall,Clean title,year,make,model,listing_date,total_price_change,price_change_percentage
0,https://www.cars.com/vehicledetail/5ee3ae04-69...,2018 Hyundai Santa Fe Sport 2.4L,"79,497 mi.","$17,899",Good Deal $2 under,Nightfall Blue,Beige,Front-wheel Drive,21-27,Gasoline,...,Yes,No,At least 1 open recall reported,NaN,2018,Hyundai,Santa Fe Sport 2.4L,2021-09-21,-2400.0,-11.823243
1,https://www.cars.com/vehicledetail/c99e5035-76...,2017 RAM 1500 Express,"80,326 mi.","$27,900",This is a good deal.,Silver,Black,Four-wheel Drive,16-23,Gasoline,...,No,Yes,NaN,NaN,2017,RAM,1500 Express,2022-03-15,-3085.0,-9.956431
2,https://www.cars.com/vehicledetail/4e79fa3a-ba...,2019 Nissan Kicks SR,"92,959 mi.","$18,950",This is a fair deal. Why?,Monarch Orange / Super Black,Charcoal,Front-wheel Drive,31-36,Gasoline,...,No,Yes,NaN,NaN,2019,Nissan,Kicks SR,2022-04-16,-1030.0,-5.155155
3,https://www.cars.com/vehicledetail/86a0d096-32...,2019 RAM 1500 Laramie,"59,176 mi.","$39,950",Great Deal $968 under,Gray,Black,Four-wheel Drive,19-24,Gasoline,...,No,No,At least 1 open recall reported,NaN,2019,RAM,1500 Laramie,2022-06-01,-573.0,-1.414012
4,https://www.cars.com/vehicledetail/36efbef5-f2...,2015 Jeep Wrangler Unlimited Rubicon,"135,385 mi.","$26,950",This is a good deal.,Blue,Black,Four-wheel Drive,16-21,Gasoline,...,No,Yes,At least 1 open recall reported,NaN,2015,Jeep,Wrangler Unlimited Rubicon,2022-01-21,-3000.0,-10.016694


In [111]:
df.shape

(6002, 28)

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6002 entries, 0 to 6001
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   link                     6002 non-null   object        
 1   listing_title            6002 non-null   object        
 2   listing_mileage          6002 non-null   object        
 3   primary_price            6002 non-null   object        
 4   deal_gauge               5524 non-null   object        
 5   exterior_color           6002 non-null   object        
 6   interior_color           6002 non-null   object        
 7   drivetrain               6002 non-null   object        
 8   mpg                      5957 non-null   object        
 9   fuel_type                6002 non-null   object        
 10  transmission             6002 non-null   object        
 11  engine                   6002 non-null   object        
 12  vin                      6002 non-

In [117]:
# Check unique values to see what we're working with: 
# engine, transmission, drivetrain. 
# might need to standardize for the model to use.

def print_unqiues():

    unique_engines = sorted(df['engine'].unique())
    unique_transmissions = sorted(df['transmission'].unique())
    unique_drivetrains = sorted(df['drivetrain'].unique())
    unique_fuel_types = sorted(df['fuel_type'].unique())

    # Print the unique values
    print("Sorted Unique Engines:")
    for engine in unique_engines:
        print(engine)
    # Want to isolate: 
    # cylinders
    # liters
    # boosted ("Turbo"/"Supercharge") T/F

    print("\nSorted Unique Transmissions:")
    for transmission in unique_transmissions:
        print(transmission)
    # Not worth the effort to differentiate automatic versus manual

    print("\nSorted Unique Drivetrains:")
    for drivetrain in unique_drivetrains:
        print(drivetrain)
    # Doesn't need to be standardized, only a few values

    # Verify that fuel_type is always Gasoline
    print("\nSorted Unique Fuel Types:")
    for fuel_type in unique_fuel_types:
        print(fuel_type)
    # Only Gasoline, column can be removed.

#print_unqiues()

In [114]:
# isolate important engine attributes: 
# cylinders
# liters
# boosted T/F

import pandas as pd
import re
df = pd.read_csv('cars.csv')

# Define regular expressions for extracting attributes
#cylinders_pattern_1 = re.compile(r'(\d+)\s*Cyl')
cylinders_pattern_1 = re.compile(r'(\d+)[\s-]*Cyl')
#cylinders_pattern_2 = re.compile(r'[IVHW]-?([3-9]|1[0-2])')
cylinders_pattern_2 = re.compile(r'(?:[IVHW]|VR)-?([3-9]|1[0-2])')
liters_pattern_1 = re.compile(r'(\d+(?:\.\d+)?)L') # For cases like "3.6L"
liters_pattern_2 = re.compile(r'(\d+(?:\.\d+)?)\s*Liter') # For cases like "3.6 Liter"

boosted_pattern = re.compile(r'Turbo|Supercharge', re.IGNORECASE)

# Function to extract engine attributes
def extract_attributes(engine_str):
    # Extracting cylinders
    cylinders_match_1 = re.search(cylinders_pattern_1, engine_str)
    cylinders_match_2 = re.search(cylinders_pattern_2, engine_str)

    if cylinders_match_1:
        cylinders = cylinders_match_1.group(1)
    elif cylinders_match_2:
        cylinders = cylinders_match_2.group(1) or cylinders_match_2.group(2)
    else:
        cylinders = None

    
    #liters = re.search(liters_pattern, engine_str).group(1) if re.search(liters_pattern, engine_str) else None
    liters_match_1 = re.search(liters_pattern_1, engine_str)
    liters_match_2 = re.search(liters_pattern_2, engine_str)
    if liters_match_1:
        liters = liters_match_1.group(1)
    elif liters_match_2:
        liters = liters_match_2.group(1)
    else:
        liters = None

    # Handle the case where the liters are specified without 'L'
    if not liters and engine_str.replace('.', '', 1).isdigit():
        liters = engine_str

    boosted = bool(re.search(boosted_pattern, engine_str))
    
    return cylinders, liters, boosted

# Apply the function to the 'engine' column
df[['Cylinders', 'Liters', 'Boosted']] = df['engine'].apply(extract_attributes).apply(pd.Series)

# Display the resulting DataFrame
#pd.set_option('display.max_rows', None)
#pd.reset_option('display.max_rows')
df[['engine', 'Cylinders', 'Liters', 'Boosted']].head()

,engine,Cylinders,Liters,Boosted
0,2.4L I4 16V GDI DOHC,4,2.4,False
1,5.7L V8 16V MPFI OHV,8,5.7,False
2,1.6L I4 16V MPFI DOHC,4,1.6,False
3,5.7L V8 16V MPFI OHV,8,5.7,False
4,3.6L V6 24V MPFI DOHC,6,3.6,False
...,...,...,...,...
5997,3.5L V6 24V PDI DOHC,6,3.5,False
5998,1.6L I4 16V MPFI DOHC,4,1.6,False
5999,3.5L V6 24V PDI DOHC Twin Turbo,6,3.5,True
6000,2.5L I4 16V GDI DOHC,4,2.5,False


In [115]:
# Display unique engines along with extracted attributes in a new DataFrame
unique_engines = df['engine'].unique()
engine_data = []

for engine in unique_engines:
    cylinders, liters, boosted = extract_attributes(engine)
    engine_data.append({'Engine': engine, 'Cylinders': cylinders, 'Liters': liters, 'Boosted': boosted})

engine_df = pd.DataFrame(engine_data)
#pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')
engine_df.head()

# Edge cases:
# row 295 "Electric"
# row 391 "-"

,Engine,Cylinders,Liters,Boosted
0,2.4L I4 16V GDI DOHC,4,2.4,False
1,5.7L V8 16V MPFI OHV,8,5.7,False
2,1.6L I4 16V MPFI DOHC,4,1.6,False
3,3.6L V6 24V MPFI DOHC,6,3.6,False
4,3.0L V6 24V GDI DOHC Twin Turbo,6,3.0,True


In [116]:
df.to_csv('new_cars.csv',index=False)